<a href="https://colab.research.google.com/github/mehang/Improvised-CNN/blob/master/notebook/Caltech-256-128/resized/15*15/caltech_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("jpt")

jpt


In [ ]:
# Importing the Keras libraries and packages
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Basic ResNet Building Block 
def resnet_layer(inputs, num_filters = 16, kernel_size = 3, strides = 1, 
                 activation ='relu', batch_normalization = True, 
    conv = Conv2D(num_filters, kernel_size = kernel_size, strides = strides, 
                  padding ='same', kernel_initializer ='he_normal', 
                  kernel_regularizer = l2(1e-4)) 
  
    x = inputs 
    if conv_first: 
        x = conv(x) 
        if batch_normalization: 
            x = BatchNormalization()(x) 
        if activation is not None: 
            x = Activation(activation)(x) 
    else: 
        if batch_normalization: 
            x = BatchNormalization()(x) 
        if activation is not None: 
            x = Activation(activation)(x) 
        x = conv(x) 
    return x

def _residual_block_first(self, x, out_channel, strides, name="unit"):
    in_channel = x.get_shape().as_list()[-1]
    with tf.variable_scope(name) as scope:
        print('\tBuilding residual unit: %s' % scope.name)

        # Shortcut connection
        if in_channel == out_channel:
            if strides == 1:
                shortcut = tf.identity(x)
            else:
                shortcut = tf.nn.max_pool(x, [1, strides, strides, 1], [1, strides, strides, 1], 'VALID')
        else:
            shortcut = self._conv(x, 1, out_channel, strides, name='shortcut')
        # Residual
        x = self._conv(x, 3, out_channel, strides, name='conv_1')
        x = self._bn(x, name='bn_1')
        x = self._relu(x, name='relu_1')
        x = self._conv(x, 3, out_channel, 1, name='conv_2')
        x = self._bn(x, name='bn_2')
        # Merge
        x = x + shortcut
        x = self._relu(x, name='relu_2')
    return x

def resnet_v1(input_shape, depth, num_classes = 10): 
      
    if (depth - 2) % 6 != 0: 
        raise ValueError('depth should be 6n + 2 (eg 20, 32, 44 in [a])') 
    # Start model definition. 
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6) 
  
    inputs = Input(shape = input_shape) 
    x = resnet_layer(inputs = inputs) 
    # Instantiate the stack of residual units 
    for stack in range(3): 
        for res_block in range(num_res_blocks): 
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack 
                strides = 2  # downsample 
            y = resnet_layer(inputs = x, 
                             num_filters = num_filters, 
                             strides = strides) 
            y = resnet_layer(inputs = y, 
                             num_filters = num_filters, 
                             activation = None) 
            if stack > 0 and res_block == 0:  # first layer but not first stack 
                # linear projection residual shortcut connection to match 
                # changed dims 
                x = resnet_layer(inputs = x, 
                                 num_filters = num_filters, 
                                 kernel_size = 1, 
                                 strides = strides, 
                                 activation = None, 
                                 batch_normalization = False) 
            x = keras.layers.add([x, y]) 
            x = Activation('relu')(x) 
        num_filters *= 2
  
    # Add classifier on top. 
    # v1 does not use BN after last shortcut connection-ReLU 
    x = AveragePooling2D(pool_size = 8)(x) 
    y = Flatten()(x) 
    outputs = Dense(num_classes, 
                    activation ='softmax', 
                    kernel_initializer ='he_normal')(y) 
  
    # Instantiate model. 
    model = Model(inputs = inputs, outputs = outputs) 
    return model


In [ ]:

def res_identity(x, filters): 
  #resnet block where dimension does not change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

  x_skip = x # this will be used for addition with the residual block 
  f1, f2 = filters

  #first block 
  x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # third block activation used after adding the input
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  # x = Activation(activations.relu)(x)

  # add the input 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

def resnet50():

  input_im = Input(shape=(train_im.shape[1], train_im.shape[2], train_im.shape[3])) # cifar 10 images size
  x = ZeroPadding2D(padding=(3, 3))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPooling2D((3, 3), strides=(2, 2))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 3rd stage

  x = res_conv(x, s=2, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))

  # 4th stage

  x = res_conv(x, s=2, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))

  # 5th stage

  x = res_conv(x, s=2, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))

  # ends with average pooling and dense connection

  x = AveragePooling2D((2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(len(class_types), activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model